# client instance 생성

In [1]:
DEPLOYMENT_URL="http://10.8.1.229:8000/"
# DEPLOYMENT_URL="http://localhost:2024/"

from langgraph_sdk import get_client

client = get_client(url=DEPLOYMENT_URL)

# thread 생성 (앨런의 channel에 대응)

- 생성 요청 후 thread_id를 포함한 데이터를 return 받음
- 하나의 thread에 대해서 여러 에이전트를 호출하고 대화가능
- 현재 앨런은 한개의 channel에 대해서 한개의 agent가 1:1로 대응되므로 현재는 고려할 필요 없지만, 추후에 1개의 채널에 대해서 여러 agent를 호출가능

In [2]:
thread = await client.threads.create()
print(thread)

thread_id = thread["thread_id"]
print(thread_id)

{'thread_id': 'fdf00c35-8d87-41a8-862f-441596648293', 'created_at': '2025-08-12T08:01:33.793709+00:00', 'updated_at': '2025-08-12T08:01:33.793709+00:00', 'metadata': {}, 'status': 'idle', 'config': {}, 'values': None, 'interrupts': {}, 'error': None}
fdf00c35-8d87-41a8-862f-441596648293


# 에이전트 호출

- 해당 thread에서 어떤 agent를 호출할 지 assistant_id를 포함하여 요청을 보냄

## parameters
- thread_id : 위에서 생성한 thread_id
- assistant_id : 앨런의 persona_id에 대응, agent를 구분하는 id, 슬라이드 에이전트의 경우 "slide_generate_agent"
- input : 해당 agent 요청에 대한 parameter, 현재는 messages만 존재, 앨런 페이지에서 유저로 부터 받는 input을 해당 parameter에 dict로 넣어줌
- stream_mode : 어떤 event에 따라서 메시지를 받을지 지정, updates(한 단계가 끝날 때마나 메시지가 옴), events(llm이 생성하는 단위로 메시지 생성)

In [3]:
list_msg_id = list()


def get_data(chunk):
    data = list(chunk.data.values())[0]
    return data


def filter_ai_msg(chunk):
    # 유저와 채팅 창에 출력될 ai 메시지를 필터링하는 함수
    if 'updates' in chunk.event:
        data = get_data(chunk)

        if data is not None:
            if 'messages' in data.keys():
                if data['messages'][-1]["name"] is not None:
                    msg_type = data['messages'][-1]['type']
                    msg_id = data['messages'][-1]['id']
                    if msg_id not in list_msg_id and msg_type == "ai":
                        list_msg_id.append(msg_id)
                        return True
    return False


def filter_silde_data(chunk):
    # slide 관련 데이터를 담은 메시지를 필터링하는 함수
    # 슬라이드 한페이지 단위로 메시지가 옴
    if 'updates' in chunk.event:
        node_name = chunk.data.keys()
        if 'executor' in node_name:
            return True

In [4]:
assistant_id = "slide_generate_agent"
list_chunk = list()

In [5]:
input = { "messages": "안녕"}

async for chunk in client.runs.stream(
    thread_id,
    assistant_id=assistant_id,
    input=input,
    stream_mode=["updates"],
    stream_subgraphs=True
):
    try:
        list_chunk.append(chunk)
        if filter_ai_msg(chunk):
            print("#" * 10 + "  AI 메시지  " + "#" * 10)
            print(chunk.data['agent']['messages'][-1]['content'])
        elif filter_silde_data(chunk):
            print("#" * 10 + "  슬라이드 데이터  " + "#" * 10)
            print(chunk)
    except:
        continue
    pass

##########  AI 메시지  ##########
안녕하세요! 슬라이드를 만드는 데 도움이 필요하신가요, 아니면 다른 질문이 있으신가요?


In [6]:
input = { "messages": "제주도 여행을 주제로 슬라이드를 만들어줘"}

async for chunk in client.runs.stream(
    thread_id,
    assistant_id=assistant_id,
    input=input,
    stream_mode=["updates"],
    stream_subgraphs=True
):
    try:
        list_chunk.append(chunk)
        if filter_ai_msg(chunk):
            print("#" * 10 + "  AI 메시지  " + "#" * 10)
            data = get_data(chunk)
            print(data['messages'][-1]['content'])
        elif filter_silde_data(chunk):
            print("#" * 10 + "  슬라이드 데이터  " + "#" * 10)
            print(chunk)
    except:
        continue
    pass

##########  AI 메시지  ##########
좋아요, 제주도 여행 슬라이드를 만들기 위해 몇 가지 정보를 더 알고 싶습니다. 몇 가지 질문 드릴게요.

1. 슬라이드의 대상 청중은 누구인가요? 예를 들어, 친구들, 가족, 회사 동료 등.
2. 슬라이드에 포함하고 싶은 주요 내용은 무엇인가요? 예를 들어, 관광 명소, 음식 문화, 여행 팁 등.
3. 슬라이드의 개수를 알고 싶습니다.
4. 특별히 강조하고 싶은 주제가 있나요?


In [7]:
input = {"messages": "알아서 해줘"}

async for chunk in client.runs.stream(
    thread_id,
    assistant_id=assistant_id,
    input=input,
    stream_mode=["updates"],
    stream_subgraphs=True
):

    if filter_ai_msg(chunk):
        print("#" * 10 + "  AI 메시지  " + "#" * 10)
        data = get_data(chunk)
        msg_id = data['messages'][-1]['id']

        print(data['messages'][-1]['content'])
    elif filter_silde_data(chunk):
        print("#" * 10 + "  슬라이드 데이터  " + "#" * 10)


##########  AI 메시지  ##########

##########  AI 메시지  ##########
검색 도구를 사용하여 목차를 생성하기 위한 조사를 시작합니다.
##########  AI 메시지  ##########
생성된 목차는 다음과 같습니다. 

2. Jeju Island Travel - This section will cover the most popular tourist destinations in Jeju, providing insights into must-visit sites, natural beauty, and unique attractions. 
3. Jeju Island Travel - This section will explore Jeju's local cuisine, highlighting traditional dishes, recommended restaurants, and unique culinary experiences specific to the island. 
4. Jeju Island Travel - This section provides guidance on planning trip itineraries in Jeju, including suggested tour routes that maximize sightseeing experiences over different durations, such as 2-day, 3-day trips. 
5. Jeju Island Travel - This section explores various accommodation choices available in Jeju, from budget to luxury, including hotels, guesthouses, and unique stays like traditional Korean houses. 
6. Jeju Island Travel - This section discusses the transportation sys

In [8]:
for chunk in list_chunk:
    if filter_silde_data(chunk):
        print("#" * 10 + "  슬라이드 데이터  " + "#" * 10)
        print(chunk.data)
        break

In [9]:
for chunk in list_chunk:
    if filter_ai_msg(chunk):
        print("#" * 10 + "  AI 메시지  " + "#" * 10)
        data = get_data(chunk)
        print(data['messages'][-1]['content'])


In [10]:
state = await client.threads.get_state(thread_id=thread_id)

for msg in state["values"]["messages"]:
    if msg["type"] != "human":
        print("#" * 10 + "  AI 메시지  " + "#" * 10)
        print(msg["content"])

##########  AI 메시지  ##########
안녕하세요! 슬라이드를 만드는 데 도움이 필요하신가요, 아니면 다른 질문이 있으신가요?
##########  AI 메시지  ##########
좋아요, 제주도 여행 슬라이드를 만들기 위해 몇 가지 정보를 더 알고 싶습니다. 몇 가지 질문 드릴게요.

1. 슬라이드의 대상 청중은 누구인가요? 예를 들어, 친구들, 가족, 회사 동료 등.
2. 슬라이드에 포함하고 싶은 주요 내용은 무엇인가요? 예를 들어, 관광 명소, 음식 문화, 여행 팁 등.
3. 슬라이드의 개수를 알고 싶습니다.
4. 특별히 강조하고 싶은 주제가 있나요?
##########  AI 메시지  ##########

##########  AI 메시지  ##########
Successfully transferred to slide_create_agent
##########  AI 메시지  ##########
검색 도구를 사용하여 목차를 생성하기 위한 조사를 시작합니다.
##########  AI 메시지  ##########
생성된 목차는 다음과 같습니다. 

2. Jeju Island Travel - This section will cover the most popular tourist destinations in Jeju, providing insights into must-visit sites, natural beauty, and unique attractions. 
3. Jeju Island Travel - This section will explore Jeju's local cuisine, highlighting traditional dishes, recommended restaurants, and unique culinary experiences specific to the island. 
4. Jeju Island Travel - This section provides guidance on planning trip itineraries

In [11]:
len(state["values"]["slides"])

14

In [12]:
state["values"]["slides"]

[{'slide_type': 'title',
  'topic': '제주도 여행',
  'idx': 0,
  'name': '타이틀 슬라이드',
  'description': '제주도 여행에 대한 프레젠테이션 타이틀 페이지',
  'requirements': ['타이틀 텍스트', '부제목 또는 설명'],
  'research': True,
  'content': '## 타이틀 슬라이드\n\n제주도 여행 (Jeju Island Travel) Presentation\n\n**Overview:** \nJeju Island, often referred to as the "Hawaii of South Korea," stands as the country\'s largest volcanic island, recognized as a UNESCO World Natural Heritage site due to its unique geological features. Known for its captivating landscapes, the island offers diverse attractions such as picturesque beaches including Iho Tewoo Beach and Samyang Beach, the Olle Trails, and the renowned Hallasan Mountain, which is the tallest mountain in South Korea [1][2]. \n\n**제주도의 주요 특징:**\n\n- **언제 방문할까?**: 보통 봄과 여름이 제주도를 방문하기 좋은 시기입니다. 특히 봄의 끝자락과 여름은 제주도를 방문하기에 좋은 계절로 알려져 있습니다 [3].\n\n- **유네스코 세계자연유산**: 제주도는 독특한 화산 지형과 다양한 자연 경관 덕분에 유네스코 세계자연유산에 등재되었습니다. \n\n- **주요 관광지**: 우도, 한라산, 만장굴, 한라산 국립공원 및 용머리 해안과 같은 유명한 여행지가 사랑받고 있습니다 

In [13]:
for s in state["values"]["slides"]:
    print(s["idx"])

0
1
2
3
4
5
6
0
1
2
3
4
5
6


In [15]:
state["values"]

{'messages': [{'content': '안녕',
   'additional_kwargs': {},
   'response_metadata': {},
   'type': 'human',
   'name': None,
   'id': 'e2f3e9a4-3f32-46ad-9c29-900e8673da58',
   'example': False},
  {'content': '안녕하세요! 슬라이드를 만드는 데 도움이 필요하신가요, 아니면 다른 질문이 있으신가요?',
   'additional_kwargs': {'refusal': None},
   'response_metadata': {'token_usage': {'completion_tokens': 27,
     'prompt_tokens': 118,
     'total_tokens': 145,
     'completion_tokens_details': {'accepted_prediction_tokens': 0,
      'audio_tokens': 0,
      'reasoning_tokens': 0,
      'rejected_prediction_tokens': 0},
     'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}},
    'model_name': 'gpt-4o-2024-08-06',
    'system_fingerprint': 'fp_ee1d74bde0',
    'id': 'chatcmpl-C3eEolQ7WujJ5D9Ex345Oc841KpN7',
    'service_tier': None,
    'finish_reason': 'stop',
    'logprobs': None,
    'content_filter_results': {}},
   'type': 'ai',
   'name': 'supervisor',
   'id': 'run--b3776c3c-8c9b-40ba-99d8-3c5f32814e3c-0'